In [ ]:
import os
import pyparsing as pp
import pandas as pd
import re


In [24]:
main_path = 'Amsterdam_input'
output_path = os.path.join(main_path,"parsed")
if not os.path.isdir(output_path):
    os.mkdir(output_path)

In [45]:
def parse_dat(main_path, dat, dfs = None, _to_csv = True):
    nested_braces = pp.nestedExpr('{', '}')
    path = os.path.join(main_path,dat)
    f = open(path) # file to be parsed
    t = None # table to be stored into csv, pandas
    if dfs is None:
        dfs = dict()
    temp = None # tempoarary record holder (for multiline records)
    table_name = None
    for l in f.readlines():
        # new table condition
        if l.strip() and not l.startswith("{") and not l.startswith('format') and temp is None:
            if table_name is not None and t is not None:
                # if we already have table
                df = pd.DataFrame.from_records(t)
                if _to_csv:
                    df.to_csv(os.path.join(output_path, dat[:-4] + "_" + table_name +".csv"))
                print("\t",len(t), " records parsed.")
                t = None
                dfs[table_name] = df
            table_name = l.split(":")[0]
            print(table_name, " table found in file.")
            new_table_flag = True
        if new_table_flag and l.startswith("{"):
            t = list()
            new_table_flag = False
        if t is not None:
            if l.startswith("{"):
                if len(l.split("}")[-1].replace("\n","").replace("\t",""))==0:
                    if l.count("{") == l.count("}"):
                        temp = l 
                        t.append(nested_braces.parseString(temp).asList()[0]) 
                        temp= None
                    else:
                        temp = l if temp is None else temp+l
                else:
                    temp = l if temp is None else temp+l
    # to capture last table of parsed file
    if t is not None:
        df = pd.DataFrame.from_records(t)
        if _to_csv:
            df.to_csv(os.path.join(output_path, dat[:-4] + "_" + table_name +".csv"))
        print("\t",len(t), " records parsed.")
        dfs[table_name] = df
    return dfs

In [46]:
dfs = parse_dat(main_path,'transit_network.dat')

stops  table found in file.
	 104  records parsed.
stops_distances  table found in file.
	 8  records parsed.
travel_time_disruptions  table found in file.


In [ ]:
dfs = dict()
for dat in os.listdir(main_path):
    try:
        df = pd.read_table(os.path.join(main_path,dat))
        dfs[dat[:-4]]= df
        df.to_csv(os.path.join(main_path,dat).replace(".dat",".csv"))
        print("Loaded {} records on {} columns from {} file".format(df.shape[0],df.shape[1],dat))
    except:
        print("Failed to load: ",dat)